In [ ]:
import pandas as pd
from networkx import *
import numpy as np
import networkx as nx
from itertools import combinations
from collections import Counter
import time
import pickle
import os
!pip install powerlaw
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
file ="/content/drive/MyDrive/NI_Research/NI_Paper-Author_with_attributes_exp5.csv"
paper_author = pd.read_csv(file, keep_default_na=False, encoding = 'utf-8',low_memory=False)
#u = paper_author.select_dtypes(object)
#paper_author[u.columns] = u.apply(
#    lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

In [ ]:
paper_author

In [ ]:
##check formats of columns
paper_author = paper_author.convert_dtypes()
paper_author[['Paper_Year','Author_NI_StartDate', 'Author_NI_EndDate', 'Author_YearJoinedUniversity', 'Author_YearDegree', 'Author_Department1', 'Paper_PubVenue_combined']].dtypes

In [ ]:
## convert year column to int
paper_author['Author_YearDegree'] = pd.to_numeric(paper_author['Author_YearDegree'], errors = 'coerce')
paper_author['Author_YearDegree'] = paper_author['Author_YearDegree'].fillna(0).astype(int)
paper_author['Author_NI_StartDate'] = pd.to_numeric(paper_author['Author_NI_StartDate'], errors = 'coerce')
paper_author['Author_NI_StartDate'] = paper_author['Author_NI_StartDate'].fillna(0).astype(int)
paper_author['Author_NI_EndDate'] = pd.to_numeric(paper_author['Author_NI_EndDate'], errors = 'coerce')
paper_author['Author_NI_EndDate'] = paper_author['Author_NI_EndDate'].fillna(0).astype(int)
paper_author['Author_YearJoinedUniversity'] = pd.to_numeric(paper_author['Author_YearJoinedUniversity'], errors = 'coerce')
paper_author['Author_YearJoinedUniversity'] = paper_author['Author_YearJoinedUniversity'].fillna(0).astype(int)
paper_author['Author_NI'] = pd.to_numeric(paper_author['Author_NI'], errors = 'coerce')
paper_author['Author_NI'] = paper_author['Author_NI'].fillna(0).astype(int)
paper_author['Paper_Year'] = pd.to_numeric(paper_author['Paper_Year'], errors = 'coerce')
paper_author['Paper_Year'] = paper_author['Paper_Year'].fillna(0).astype(int)
paper_author['Author_subset_exp5'] = pd.to_numeric(paper_author['Author_subset_exp5'], errors = 'coerce')
paper_author['Author_subset_exp5'] = paper_author['Author_subset_exp5'].fillna(0).astype(int)
#paper_author['Author_Department1'] = paper_author['Author_Department1'].astype(str)

In [ ]:
paper_author[['Author_subset_exp5','Paper_Year','Author_NI_StartDate', 'Author_NI_EndDate', 'Author_YearJoinedUniversity', 'Author_YearDegree', 'Author_Department1','Paper_PubVenue_combined']].dtypes

In [ ]:
## must be imported as nx.DiGraph to preserve order of source,target (paper,author)
paper_author_net = nx.from_pandas_edgelist(paper_author, source='Source', target='Target', edge_attr=True, create_using=nx.DiGraph())
#undirected version
#paper_author_net_undirected = nx.from_pandas_edgelist(paper_author, source='Source', target='Target', edge_attr=True, create_using=nx.OrderedGraph())

In [ ]:
# Add node attributes
nodelist= pd.read_csv('/content/drive/MyDrive/NI_Research/NI_Author_Nodelist_exp5.csv')
u2 = nodelist.select_dtypes(object)
nodelist[u2.columns] = u2.apply(
    lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

In [ ]:
for i, nlrow in nodelist.iterrows():
  try:
    paper_author_net.nodes[nlrow['Target']].update(nlrow[1:].to_dict())
  except:
    continue
print(nlrow)

In [ ]:
nodelist = nodelist.convert_dtypes()

In [ ]:
nodelist['Author_YearJoinedUniversity'] = pd.to_numeric(nodelist['Author_YearJoinedUniversity'], errors = 'ignore')
nodelist['Author_YearJoinedUniversity'] = nodelist['Author_YearJoinedUniversity'].fillna(0).astype(int)
nodelist['Author_YearDegree'] = pd.to_numeric(nodelist['Author_YearDegree'], errors = 'ignore')
nodelist['Author_YearDegree'] = nodelist['Author_YearDegree'].fillna(0).astype(int)
nodelist['Author_NI_StartDate'] = pd.to_numeric(nodelist['Author_NI_StartDate'], errors = 'ignore')
nodelist['Author_NI_StartDate'] = nodelist['Author_NI_StartDate'].fillna(0).astype(int)
nodelist['Author_NI_EndDate'] = pd.to_numeric(nodelist['Author_NI_EndDate'], errors = 'ignore')
nodelist['Author_NI_EndDate'] = nodelist['Author_NI_EndDate'].fillna(0).astype(int)
nodelist['Author_NI'] = pd.to_numeric(nodelist['Author_NI'], errors = 'ignore')
nodelist['Author_NI'] = nodelist['Author_NI'].fillna(0).astype(int)
#nodelist['Author_Department1'] = nodelist['Author_Department1'].astype(str)
nodelist['Author_subset_exp5'] = pd.to_numeric(nodelist['Author_subset_exp5'], errors = 'ignore')
nodelist['Author_subset_exp5'] = nodelist['Author_subset_exp5'].fillna(0).astype(int)

In [ ]:
nodelist[['Author_YearJoinedUniversity','Author_subset_exp5','Author_NI_StartDate','Author_NI_EndDate', 'Author_YearDegree', 'Author_Department1']].dtypes

In [ ]:
NI_only_nodelist = pd.read_csv('/content/drive/MyDrive/NI_Research/Exp5_ego_intersection.csv')

# Experiments

**Experiment 5:** 4 years publications before & after + *0* year gap between before-after

In [ ]:
# Step 1: subset the network to include nodes with appropriate YearDegree & YearJoin
exp5_subset = [(u,v) for u,v,e in paper_author_net.edges(data=True) if e['Author_NI_StartDate'] <= 2021-4 and e['Author_YearDegree'] <= e['Author_NI_StartDate']-4 and e['Author_NI_StartDate'] != 0 and e['Author_YearDegree'] != 0 and e['Author_NI_StartDate'] != 9999 and e['Author_YearDegree'] != 9999 and e['Author_Department1'] != '9999' and e['Paper_PubVenue_combined'] != '0' and e['Author_subset_exp5'] == 1]
exp5_subset_df = pd.DataFrame(exp5_subset, columns = ['Source','Target'])
exp5_subset_attributes = exp5_subset_df.merge(paper_author,how='left',on=('Source','Target'))
exp5_subset_net = nx.from_pandas_edgelist(exp5_subset_attributes, source='Source', target='Target',edge_attr=True, create_using=nx.DiGraph())

In [ ]:
#Step 2: preparation of BEFORE-NI: subset paper-author edges with appropriate pub range & gap size before-after
exp5_before = [(u,v) for u,v,e in exp5_subset_net.edges(data=True) if e['Author_NI_StartDate'] - e['Paper_Year'] <= 4 and e['Author_NI_StartDate'] - e['Paper_Year'] >= 1 and e['Author_NI_StartDate'] != 0 and e['Paper_Year'] != 0 and e['Author_NI_StartDate'] != 9999 and e['Author_YearDegree'] != 9999]
exp5_before_df = pd.DataFrame(exp5_before, columns = ['Source','Target'])
exp5_before_attributes = exp5_before_df.merge(paper_author,how='left',on=('Source','Target'))
exp5_before_attributes.to_csv('exp5_before_paper_author_attributes.csv')

In [ ]:
# Step 3: projection of BEFORE NI network
# Before - ALL (not limited to NI-NI edges)
exp5_before_net = nx.from_pandas_edgelist(exp5_before_attributes, source='Source', target='Target', edge_attr=True, create_using=nx.Graph())

In [ ]:
source_nodelist_before = exp5_before_attributes.Source ## papers only
target_nodelist_before = exp5_before_attributes.Target  ## for authors only
## unique list of attributes for source and target
from collections import OrderedDict
source_nodelist_before_sorted = sorted(list(OrderedDict.fromkeys(source_nodelist_before)))
target_nodelist_before_sorted = sorted(list(OrderedDict.fromkeys(target_nodelist_before)))

In [ ]:
len(source_nodelist_before_sorted) ## 1861 initially, but down to 1758 unique papers

In [ ]:
subset_exp5_before = paper_author[paper_author["Source"].isin(source_nodelist_before)]
subset_exp5_before.shape[0] ## total paper-author edges based on 1769 unique papers

In [ ]:
subset_exp5_before.to_csv('exp5_before_NI_all_edges.csv')

In [ ]:
exp5_before_all_net = nx.from_pandas_edgelist(subset_exp5_before, source='Source', target='Target', edge_attr=True, create_using=nx.Graph())
target_before_full_sorted = sorted(list(OrderedDict.fromkeys(subset_exp5_before['Target'])))

In [ ]:
before_projection_all = bipartite.weighted_projected_graph(exp5_before_all_net, target_before_full_sorted, ratio=False)


In [ ]:
## PAPER projection
before_projection_paper_paper = bipartite.weighted_projected_graph(exp5_before_all_net, source_nodelist_before_sorted, ratio=False)

In [ ]:
number_of_nodes(before_projection_all)

In [ ]:
number_of_edges(before_projection_all)

In [ ]:
nx.write_weighted_edgelist(before_projection_all, "exp5_before_projection.csv", delimiter=',')
paper_edgelist = pd.read_csv('exp5_before_projection.csv',names=["Source","Target","Weight"])
paper_edgelist.to_csv('exp5_before_projection.csv')

In [ ]:
sorted(list(before_projection_all.degree(list(target_nodelist_before_sorted))))

In [ ]:
len(before_projection_all)

In [ ]:
## NI-NI ties only
before_NI_only = before_projection_all.subgraph(NI_only_nodelist['Target'])
sorted(list(before_NI_only.degree(list(target_nodelist_before_sorted))))

In [ ]:
number_of_nodes(before_NI_only)

In [ ]:
number_of_edges(before_NI_only)

In [ ]:
nx.write_weighted_edgelist(before_NI_only, "exp5_before_NI_only_projection.csv", delimiter=',')
paper_edgelist = pd.read_csv('exp5_before_NI_only_projection.csv',names=["Source","Target","Weight"])
paper_edgelist.to_csv('exp5_before_NI_only_projection.csv')

In [ ]:
before_all_list = pd.DataFrame(sorted(list(before_projection_all.degree(list(target_nodelist_before_sorted)))), columns = ['node','count'])

## NI-NI ties only
before_NI_only = before_projection_all.subgraph(NI_only_nodelist['Target'])
before_NI_list = pd.DataFrame(sorted(list(before_NI_only.degree(list(target_nodelist_before_sorted)))), columns = ['node','count'])

exp5_before_egonets = before_all_list.merge(before_NI_list, how='inner', on='node')
exp5_before_egonets.to_csv("exp5_before_egonets.csv")

In [ ]:
#paper-centric: get number of authors for each paper
source_before_full_sorted = sorted(list(OrderedDict.fromkeys(subset_exp5_before['Source'])))
degreeView = exp5_before_all_net.degree(source_before_full_sorted)
degreeView = pd.DataFrame(degreeView, columns=['paper_name','num_coauthors'])
degreeView.to_csv('exp5_before_paper_counts.csv',header=['paper_name','num_coauthors'])
#degree_counts = Counter(dict(degreeView))

print("\n----------- Mean -----------\n")
print(degreeView['num_coauthors'].mean())
print("\n----------- Median -----------\n")
print(degreeView['num_coauthors'].median())
print("\n-----------  Mode -----------\n")
print(degreeView['num_coauthors'].mode())
print("\n-----------  Stats -----------\n")
print(degreeView['num_coauthors'].describe())
print("\n-----------  Skew -----------\n")
print(degreeView['num_coauthors'].skew())

In [ ]:
# paper-centric: get number of authors for each paper
source_before_full_sorted = sorted(list(OrderedDict.fromkeys(subset_exp5_before['Source'])))
degreeView = exp5_before_all_net.degree(source_before_full_sorted)
degreeView = pd.DataFrame(degreeView, columns=['paper_name','num_coauthors'])
degreeView.to_csv('exp5_before_paper_counts.csv',header=['paper_name','num_coauthors'])
#degree_counts = Counter(dict(degreeView))

print("\n----------- Mean -----------\n")
print(degreeView['num_coauthors'].mean())
print("\n----------- Median -----------\n")
print(degreeView['num_coauthors'].median())
print("\n-----------  Mode -----------\n")
print(degreeView['num_coauthors'].mode())
print("\n-----------  Stats -----------\n")
print(degreeView['num_coauthors'].describe())
print("\n-----------  Skew -----------\n")
print(degreeView['num_coauthors'].skew())

In [ ]:
degreeView.to_csv("before_unique papers & authors counts.csv")

In [ ]:
## ego-centric: for each ego, print # publications before vs after
ego_pub_all = pd.DataFrame(exp5_before_all_net.degree(target_nodelist_before_sorted), columns=['author_name','num_papers_all'])
ego_pub_coauthors_all = pd.DataFrame(before_projection_all.degree(target_nodelist_before_sorted), columns = ['author_name','co_authors_all'])
ego_pub_coauthors_NIonly = pd.DataFrame(before_NI_only.degree(target_nodelist_before_sorted), columns = ['author_name','co_authors_NIonly'])

ego_pub_final = pd.DataFrame()
ego_pub_final['author_name'] = ego_pub_all['author_name']
ego_pub_final['num_papers'] = ego_pub_all['num_papers_all']
ego_pub_final['num_coauthors_all'] = ego_pub_coauthors_all['co_authors_all']
ego_pub_final['num_coauthors_NIonly'] = ego_pub_coauthors_NIonly['co_authors_NIonly']
ego_pub_final.to_csv('exp5_before_author_counts.csv',header=['author_name', 'num_papers', 'num_coauthors_all', 'num_coauthors_NIonly'])

print("\n----------- Mean -----------\n")
print(ego_pub_final[['num_papers','num_coauthors_all','num_coauthors_NIonly']].mean())
print("\n----------- Median -----------\n")
print(ego_pub_final[['num_papers','num_coauthors_all','num_coauthors_NIonly']].median())
print("\n-----------  Mode -----------\n")
print(ego_pub_final[['num_papers','num_coauthors_all','num_coauthors_NIonly']].mode())
print("\n-----------  Stats -----------\n")
print(ego_pub_final[['num_papers','num_coauthors_all','num_coauthors_NIonly']].describe())
print("\n-----------  Skew -----------\n")
print(ego_pub_final[['num_papers','num_coauthors_all','num_coauthors_NIonly']].skew())


In [ ]:
ego_pub_all.to_csv("before_num_papers.csv")

In [ ]:
ego_pub_final['num_papers'].sum() ## not unique though, unique is above in count

In [ ]:
 ## #Step 4: preparation of AFTER-NI: subset paper-author edges with appropriate pub range & gap size before-after
exp5_after = [(u,v) for u,v,e in exp5_subset_net.edges(data=True) if e['Paper_Year'] - e['Author_NI_StartDate'] <= 4 and e['Paper_Year'] - e['Author_NI_StartDate'] >= 1 and e['Author_NI_StartDate'] != 0 and e['Paper_Year'] != 0 and e['Author_NI_StartDate'] != 9999 and e['Author_YearDegree'] != 9999]
exp5_after_df = pd.DataFrame(exp5_after, columns = ['Source','Target'])
exp5_after_attributes = exp5_after_df.merge(paper_author,how='left',on=('Source','Target'))
exp5_after_attributes.to_csv('exp5_after_paper_author_attributes.csv')

In [ ]:
# Step 5: projection of AFTER NI network
# AFTER - ALL (not limited to NI-NI edges)
exp5_after_net = nx.from_pandas_edgelist(exp5_before_attributes, source='Source', target='Target', edge_attr=True, create_using=nx.Graph())

In [ ]:
source_nodelist_after = exp5_after_attributes.Source ## papers only
target_nodelist_after = exp5_after_attributes.Target  ## for authors only
## unique list of attributes for source and target
from collections import OrderedDict
source_nodelist_after_sorted = sorted(list(OrderedDict.fromkeys(source_nodelist_after)))
target_nodelist_after_sorted = sorted(list(OrderedDict.fromkeys(target_nodelist_after)))

In [ ]:
len(source_nodelist_after_sorted) ## unique papers

In [ ]:
subset_exp5_after = paper_author[paper_author["Source"].isin(source_nodelist_after)]
subset_exp5_after = subset_exp5_after[subset_exp5_after['Author_Department1'] != '9999']   ### added July 19
subset_exp5_after.shape[0] ## total paper-author edges based on unique papers

In [ ]:
subset_exp5_after.to_csv('exp5_after_NI_all_edges.csv')

In [ ]:
exp5_after_all_net = nx.from_pandas_edgelist(subset_exp5_after, source='Source', target='Target', edge_attr=True, create_using=nx.Graph())
target_after_full_sorted = sorted(list(OrderedDict.fromkeys(subset_exp5_after['Target'])))

In [ ]:
after_projection_all = bipartite.weighted_projected_graph(exp5_after_all_net, target_after_full_sorted, ratio=False)
#list(before_projection_all.edges(data=True))

In [ ]:
##Paper Projection
after_projection_paper_paper = bipartite.weighted_projected_graph(exp5_after_all_net, source_nodelist_after_sorted, ratio=False)

In [ ]:
number_of_nodes(after_projection_all) ## how many authors (all, not just NI)

In [ ]:
number_of_edges(after_projection_all)

In [ ]:
nx.write_weighted_edgelist(before_projection_all, "exp5_after_projection.csv", delimiter=',')
paper_edgelist = pd.read_csv('exp5_after_projection.csv',names=["Source","Target","Weight"])
paper_edgelist.to_csv('exp5_after_projection.csv')

In [ ]:
sorted(list(after_projection_all.degree(list(target_nodelist_after_sorted))))

In [ ]:
after_proj_list = list(after_projection_all)
after_proj_list

In [ ]:
## NI-NI ties only
after_NI_only = after_projection_all.subgraph(NI_only_nodelist['Target'])
sorted(list(after_NI_only.degree(list(target_nodelist_after_sorted))))

In [ ]:
number_of_nodes(after_NI_only)

In [ ]:
number_of_edges(after_NI_only)

In [ ]:
nx.write_weighted_edgelist(after_NI_only, "exp5_after_NI_only_projection.csv", delimiter=',')
paper_edgelist = pd.read_csv('exp5_after_NI_only_projection.csv',names=["Source","Target","Weight"])
paper_edgelist.to_csv('exp5_after_NI_only_projection.csv')

In [ ]:
after_all_list = pd.DataFrame(sorted(list(after_projection_all.degree(list(target_nodelist_after_sorted)))), columns = ['node','count'])

## NI-NI ties only
after_NI_only = after_projection_all.subgraph(NI_only_nodelist['Target'])
after_NI_list = pd.DataFrame(sorted(list(after_NI_only.degree(list(target_nodelist_after_sorted)))), columns = ['node','count'])

exp5_after_egonets = after_all_list.merge(after_NI_list, how='inner', on='node')
exp5_after_egonets.to_csv("exp5_after_egonets.csv")

In [ ]:
## ego-centric: for each ego, print # publications before vs after
ego_pub_all = pd.DataFrame(exp5_after_all_net.degree(target_nodelist_after_sorted), columns=['author_name','num_papers_all'])
ego_pub_coauthors_all = pd.DataFrame(after_projection_all.degree(target_nodelist_after_sorted), columns = ['author_name','co_authors_all'])
ego_pub_coauthors_NIonly = pd.DataFrame(after_NI_only.degree(target_nodelist_after_sorted), columns = ['author_name','co_authors_NIonly'])

ego_pub_final = pd.DataFrame()
ego_pub_final['author_name'] = ego_pub_all['author_name']
ego_pub_final['num_papers'] = ego_pub_all['num_papers_all']
ego_pub_final['num_coauthors_all'] = ego_pub_coauthors_all['co_authors_all']
ego_pub_final['num_coauthors_NIonly'] = ego_pub_coauthors_NIonly['co_authors_NIonly']
ego_pub_final.to_csv('exp5_after_author_counts.csv',header=['author_name', 'num_papers', 'num_coauthors_all', 'num_coauthors_NIonly'])

print("\n----------- Mean -----------\n")
print(ego_pub_final[['num_papers','num_coauthors_all','num_coauthors_NIonly']].mean())
print("\n----------- Median -----------\n")
print(ego_pub_final[['num_papers','num_coauthors_all','num_coauthors_NIonly']].median())
print("\n-----------  Mode -----------\n")
print(ego_pub_final[['num_papers','num_coauthors_all','num_coauthors_NIonly']].mode())
print("\n-----------  Stats -----------\n")
print(ego_pub_final[['num_papers','num_coauthors_all','num_coauthors_NIonly']].describe())
print("\n-----------  Skew -----------\n")
print(ego_pub_final[['num_papers','num_coauthors_all','num_coauthors_NIonly']].skew())


In [ ]:
#paper-centric: get number of authors for each paper
source_after_full_sorted = sorted(list(OrderedDict.fromkeys(subset_exp5_after['Source'])))
degreeView = exp5_after_all_net.degree(source_after_full_sorted)
degreeView = pd.DataFrame(degreeView, columns=['paper_name','num_coauthors'])
degreeView.to_csv('exp5_after_paper_counts.csv',header=['paper_name','num_coauthors'])
#degree_counts = Counter(dict(degreeView))

print("\n----------- Mean -----------\n")
print(degreeView['num_coauthors'].mean())
print("\n----------- Median -----------\n")
print(degreeView['num_coauthors'].median())
print("\n-----------  Mode -----------\n")
print(degreeView['num_coauthors'].mode())
print("\n-----------  Stats -----------\n")
print(degreeView['num_coauthors'].describe())
print("\n-----------  Skew -----------\n")
print(degreeView['num_coauthors'].skew())

In [ ]:
ego_pub_final.to_csv("after_num_papers.csv")

In [ ]:
degreeView.to_csv("after_unique papers & authors counts.csv")

## EGO-LEGEL ANALYSES

## Before EGO analysis - same dept

In [ ]:
edge_df = nx.to_pandas_edgelist(before_projection_all)  ## test either before_NI_only or before_projection_all
edge_df.rename({'source':'source','target':'target'},axis=1)

In [ ]:
edge_df_attributes = edge_df.merge(nodelist,how='left',left_on=('source'), right_on='Target', suffixes=('','_source')).merge(nodelist, how='left',left_on=('target'), right_on='Target', suffixes=('_source','_target')) ## the suffixes changes only when there's overlapping columns
edge_df_attributes

In [ ]:
before_ego_nets = []

for node in list(before_projection_all.nodes(data=True)):
    if node[0] in target_nodelist_before_sorted:
      ego = ego_graph(before_projection_all, node[0], radius=1, center=True, undirected=False, distance=None)
      ego.remove_edges_from(list(nx.selfloop_edges(ego)))
      before_ego_nets.append((node[0], ego))
    print(node)

In [ ]:
combined_df = pd.DataFrame()

## network computation in this loop

for ego in before_ego_nets:
  print(ego[0],ego[1].nodes) ## printing out the alters of the egonet
  #  attr = edge_df_attributes[(edge_df_attributes.source==ego[0]) & (edge_df_attributes.target==target)]
  #  print(attr)
  attr = edge_df_attributes[(edge_df_attributes.source==ego[0]) & (edge_df_attributes.target.isin(ego[1].nodes))].append(edge_df_attributes[(edge_df_attributes.target==ego[0]) & (edge_df_attributes.source.isin(ego[1].nodes))]) ## compare if source and target is in nodes
  ego_list = list(ego[1].nodes)
  ego_list.remove(ego[0])  ## remove selfloop , need to convert to list to not impact the network object
  attr['ego_key'] = str((ego[0],ego_list))
  combined_df = combined_df.append(attr)
#  print(attr)
  attr.to_csv(ego[0] + 'ego.csv')
combined_df.to_csv('before_egonet.csv')

In [ ]:
## Shannon diversity for Before-NI
combined_df = pd.DataFrame()

##network computation in this loop

for ego in before_ego_nets:
  print(ego[0],ego[1].nodes) ## printing out the alters of the egonet
  #  attr = edge_df_attributes[(edge_df_attributes.source==ego[0]) & (edge_df_attributes.target==target)]
  #  print(attr)
  attr = edge_df_attributes[(edge_df_attributes.source==ego[0]) & (edge_df_attributes.target.isin(ego[1].nodes))].append(edge_df_attributes[(edge_df_attributes.target==ego[0]) & (edge_df_attributes.source.isin(ego[1].nodes))]) ## compare if source and target is in nodes
  source_dept = attr[["source", "Author_Department1_source"]].drop_duplicates().reset_index(drop=True)
  target_dept = attr[["target", "Author_Department1_target"]].drop_duplicates().reset_index(drop=True)
  source_dept.columns = ["node","department"]
  target_dept.columns = ["node","department"]
  combined_dept = source_dept.append(target_dept)
 # print(combined_dept.groupby("department").count())
  ## Shannon diversity, richness, evenness
  counts = combined_dept.groupby("department").count()
  richness = combined_dept.groupby("department").nunique().reset_index(drop=True)
  counts["alters_have_dept"] = (counts["node"].sum())
  counts["total_n_alters"] = len(ego[1].nodes) - 1
  counts["alters_have_dept / all_alters"] = (counts["node"].sum()) / (len(ego[1].nodes) - 1)
  counts["simpson_per_node"] =  ((counts["node"] * (counts["node"] - 1))  / ((counts["node"].sum()) * (counts["node"].sum() - 1)))  ## code added July 20
  counts["simpson_diversity"] = 1 - counts["simpson_per_node"].sum()  ## July 20 --- Gini-Simpson index  ( 1 - Simpson so that the higher the number, the higher the diversity)
  counts["richness"] = richness["node"].count()
  counts["shannon_per_node"] = - ((counts["node"] / counts["node"].sum()) * (np.log(counts["node"] / counts["node"].sum())))   ## put shannon formula here
  counts["shannon_diversity"] = counts["shannon_per_node"].sum()
  counts["shannon_equitability"] = (counts["shannon_per_node"].sum()) / (np.log(richness["node"].count())) # Divide Shannon’s diversity index by natural logarithm of species richness ln(S) to calculate the Pielou evenness
  ##Simpson equitability
  counts["simpson_equitability"] = (counts["simpson_per_node"].sum()) / (np.log(richness["node"].count())) # Divide Simpson’s diversity index by natural logarithm of species richness ln(S) to calculate evenness
  ego_list = list(ego[1].nodes)
  ego_list.remove(ego[0])  ## remove selfloop , need to convert to list to not impact the network object
  counts['ego_key'] = str((ego[0],ego_list))
  combined_df = combined_df.append(counts)
  #print(counts)
  counts.to_csv(ego[0] + 'ego.csv')
combined_df.to_csv('before_NI_simpson & shannon.csv')

In [ ]:
#analysis on same department

combined_samedept_df = pd.DataFrame()

for ego in before_ego_nets:
  print(ego[0],ego[1].nodes) ## printing out the alters of the egonet
  #for target in ego[1].nodes:
  #  attr = edge_df_attributes[(edge_df_attributes.source==ego[0]) & (edge_df_attributes.target==target)]
  #  print(attr)
  ## make sure it is an undirected version of egonets to also include egos that end up being in the 'target' column
  attr = edge_df_attributes[(edge_df_attributes.source==ego[0]) & (edge_df_attributes.target.isin(ego[1].nodes))].append(edge_df_attributes[(edge_df_attributes.target==ego[0]) & (edge_df_attributes.source.isin(ego[1].nodes))]) ## compare if source and target is in nodes
  department_source = attr[attr['source']==ego[0]].reset_index()
  attr_department = pd.DataFrame()

  if len(department_source.index) > 0:
      dept_value = department_source.loc[0,'Author_Department1_source']
      attr2 = department_source[dept_value == department_source['Author_Department1_target']]
      attr_department = attr_department.append(attr2).drop_duplicates()
  department_target = attr[attr['target']==ego[0]].reset_index()

  if len(department_target.index) > 0:
      dept_value2 = department_target.loc[0,'Author_Department1_target']
      attr3 = department_target[dept_value2 == department_target['Author_Department1_source']]
      attr_department = attr_department.append(attr3).drop_duplicates()
  print(attr_department)

  ego_list = list(ego[1].nodes)
  ego_list.remove(ego[0])  ## remove selfloop , need to convert to list to not impact the network object
  attr_department['ego_key'] = str((ego[0],ego_list))
  combined_samedept_df = combined_samedept_df.append(attr_department)
#combined_samedept_df.to_csv('combined_dept_ego.csv')

In [ ]:
counts = combined_samedept_df.groupby('ego_key').count()[["source"]].reset_index() ## do apply to extract the source from ego_key
combined_samedept_df['ego_size'] = counts['source']
combined_samedept_df.to_csv('before_NI_egonets_samedept.csv')

In [ ]:
dept_value, dept_value2

## After EGO analysis - same dept

In [ ]:
edge_df = nx.to_pandas_edgelist(after_projection_all)  ## check either after_NI_only or after_projection_all
edge_df.rename({'source':'source','target':'target'},axis=1)

In [ ]:
edge_df_attributes = edge_df.merge(nodelist,how='left',left_on=('source'), right_on='Target', suffixes=('','_source')).merge(nodelist, how='left',left_on=('target'), right_on='Target', suffixes=('_source','_target')) ## the suffixes changes only when there's overlapping columns
edge_df_attributes

In [ ]:
after_ego_nets = []

for node in list(after_projection_all.nodes(data=True)):
    if node[0] in target_nodelist_after_sorted:
      ego = ego_graph(after_projection_all, node[0], radius=1, center=True, undirected=False, distance=None)
      after_ego_nets.append((node[0], ego))
    print(node)

In [ ]:
combined_after_df = pd.DataFrame()

## network computation in this loop

for ego in after_ego_nets:
  print(ego[0],ego[1].nodes) ## printing out the alters of the egonet
  #  attr = edge_df_attributes[(edge_df_attributes.source==ego[0]) & (edge_df_attributes.target==target)]
  #  print(attr)
  attr = edge_df_attributes[(edge_df_attributes.source==ego[0]) & (edge_df_attributes.target.isin(ego[1].nodes))].append(edge_df_attributes[(edge_df_attributes.target==ego[0]) & (edge_df_attributes.source.isin(ego[1].nodes))]) ## compare if source and target is in nodes
  ego_list = list(ego[1].nodes)
  ego_list.remove(ego[0])  ## remove selfloop , need to convert to list to not impact the network object
  attr['ego_key'] = str((ego[0],ego_list))
  combined_after_df = combined_after_df.append(attr)
  print(attr)
  attr.to_csv(ego[0] + 'ego.csv')
combined_after_df.to_csv('after_egonet.csv')

In [ ]:
#analysis on same department

after_combined_samedept_df = pd.DataFrame()

for ego in after_ego_nets:
  print(ego[0],ego[1].nodes) ## printing out the alters of the egonet
  #for target in ego[1].nodes:
  #  attr = edge_df_attributes[(edge_df_attributes.source==ego[0]) & (edge_df_attributes.target==target)]
  #  print(attr)
  ## make sure it is an undirected version of egonets to also include egos that end up being in the 'target' column
  attr = edge_df_attributes[(edge_df_attributes.source==ego[0]) & (edge_df_attributes.target.isin(ego[1].nodes))].append(edge_df_attributes[(edge_df_attributes.target==ego[0]) & (edge_df_attributes.source.isin(ego[1].nodes))]) ## compare if source and target is in nodes
  department_source = attr[attr['source']==ego[0]].reset_index()
  attr_department = pd.DataFrame()

  if len(department_source.index) > 0:
      dept_value = department_source.loc[0,'Author_Department1_source']
      attr2 = department_source[dept_value == department_source['Author_Department1_target']]
      attr_department = attr_department.append(attr2).drop_duplicates()
  department_target = attr[attr['target']==ego[0]].reset_index()

  if len(department_target.index) > 0:
      dept_value2 = department_target.loc[0,'Author_Department1_target']
      attr3 = department_target[dept_value2 == department_target['Author_Department1_source']]
      attr_department = attr_department.append(attr3).drop_duplicates()
  print(attr_department)

  ego_list = list(ego[1].nodes)
  ego_list.remove(ego[0])  ## remove selfloop , need to convert to list to not impact the network object
  attr_department['ego_key'] = str((ego[0],ego_list))
  after_combined_samedept_df = after_combined_samedept_df.append(attr_department)
#combined_samedept_df.to_csv('after_combined_dept_ego.csv')

In [ ]:
counts = after_combined_samedept_df.groupby('ego_key').count()[["source"]].reset_index() ## do apply to extract the source from ego_key
after_combined_samedept_df['ego_size'] = counts['source']
after_combined_samedept_df.to_csv('after_NI_egonets_samedept.csv')

In [ ]:
## Shannon diversity for AFTER
combined_after_df = pd.DataFrame()

## do network computation in this loop

for ego in after_ego_nets:
  print(ego[0],ego[1].nodes) ## printing out the alters of the egonet
  #  attr = edge_df_attributes[(edge_df_attributes.source==ego[0]) & (edge_df_attributes.target==target)]
  #  print(attr)
  attr = edge_df_attributes[(edge_df_attributes.source==ego[0]) & (edge_df_attributes.target.isin(ego[1].nodes))].append(edge_df_attributes[(edge_df_attributes.target==ego[0]) & (edge_df_attributes.source.isin(ego[1].nodes))]) ## compare if source and target is in nodes
  source_dept = attr[["source", "Author_Department1_source"]].drop_duplicates().reset_index(drop=True)
  target_dept = attr[["target", "Author_Department1_target"]].drop_duplicates().reset_index(drop=True)
  source_dept.columns = ["node","department"]
  target_dept.columns = ["node","department"]
  combined_dept = source_dept.append(target_dept)
 # print(combined_dept.groupby("department").count())
  ## Shannon diversity, richness, evenness
  counts = combined_dept.groupby("department").count()
  richness = combined_dept.groupby("department").nunique().reset_index(drop=True)
  counts["alters_have_dept"] = (counts["node"].sum())
  counts["total_n_alters"] = len(ego[1].nodes) - 1
  counts["alters_have_dept / all_alters"] = (counts["node"].sum()) / (len(ego[1].nodes) - 1)
  counts["simpson_per_node"] =  ((counts["node"] * (counts["node"] - 1))  / ((counts["node"].sum()) * (counts["node"].sum() - 1)))  ## code added July 20
  counts["simpson_diversity"] = 1 - counts["simpson_per_node"].sum()  ## July 20 --- Gini-Simpson index  ( 1 - Simpson so that the higher the number, the higher the diversity)
  counts["richness"] = richness["node"].count()
  counts["shannon_per_node"] = - ((counts["node"] / counts["node"].sum()) * (np.log(counts["node"] / counts["node"].sum())))   ## put shannon formula here
  counts["shannon_diversity"] = counts["shannon_per_node"].sum()
  counts["shannon_equitability"] = (counts["shannon_per_node"].sum()) / (np.log(richness["node"].count())) # Divide Shannon’s diversity index by natural logarithm of species richness ln(S) to calculate the Pielou evenness
  ##Simpson equitability
  counts["simpson_equitability"] = (counts["simpson_per_node"].sum()) / (np.log(richness["node"].count())) # Divide Simpson’s diversity index by natural logarithm of species richness ln(S) to calculate evenness
  ego_list = list(ego[1].nodes)
  ego_list.remove(ego[0])  ## remove selfloop , need to convert to list to not impact the network object
  counts['ego_key'] = str((ego[0],ego_list))
  combined_after_df = combined_after_df.append(counts)
  #print(counts)
  counts.to_csv(ego[0] + 'ego.csv')
combined_after_df.to_csv('after_NI_simpson & shannon.csv')